1. Overall process
2. Created github repository > .gitignore| LICENSE |README.md
3. Cloned the repo
4. Updated README.md
5. Created initial Folder structure
6. Created Virtual Environment
7. Install dependency
8. Design overall workflow (in workflow.ipynb)
    1. Setup and config
    2. create client
    3. create prompt template
    4. handle input message (msg-> scam or not)
    5. generate response from llm
    6. handle output(parsing)
9. Convert it into project files



In [13]:
# 1. Setup and config
from google import genai
from google.genai import types
import os
import json
import re
from dotenv import load_dotenv

load_dotenv()

#2. Create client
GEMNAI_MODEL = os.getenv("GEMINI_MODEL_NAME")
GEMNAI_API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key = GEMNAI_API_KEY)

In [7]:
# 3. create prompt template
PROMPT_TEMPLATE = """
You are a highly reliable and safety-focused AI system trained to identify potentially scammy messages.

Follow this exact structured reasoning format:
1. **Thought**: Analyze the tone, urgency, and patterns.
2. **Action**: Classify if this is likely a scam.
3. **Final Answer**: Output a structured JSON.

Return response only in this JSON format:
```
    {{
    "label": "Scam | Not Scam | Uncertain",
    "reasoning": "Your step-by-step analysis",
    "intent": Sender's intent behind the message",
    "risk_factors": ["List of red flags like urgency, bad links, etc."]
    }}
```
User Message:
{}
"""

In [8]:
# 4. User input
user_input = "Your package delivery failed. Please update your address at [www.bestwebsite.com] to avoid return to sender."
print(user_input)

Your package delivery failed. Please update your address at [www.bestwebsite.com] to avoid return to sender.


In [ ]:
#5. Generate response with llm
full_prompt = PROMPT_TEMPLATE.format(user_input)
print(full_prompt)




You are a highly reliable and safety-focused AI system trained to identify potentially scammy messages.

Follow this exact structured reasoning format:
1. **Thought**: Analyze the tone, urgency, and patterns.
2. **Action**: Classify if this is likely a scam.
3. **Final Answer**: Output a structured JSON.

Return response only in this JSON format:
```
    {
    "label": "Scam | Not Scam | Uncertain",
    "reasoning": "Your step-by-step analysis",
    "intent": Sender's intent behind the message",
    "risk_factors": ["List of red flags like urgency, bad links, etc."]
    }
```
User Message:
Your package delivery failed. Please update your address at [www.bestwebsite.com] to avoid return to sender.



In [10]:
response = client.models.generate_content(
    model = GEMNAI_MODEL,
    contents= full_prompt
)

print(response.text)

1. **Thought**: The message creates a sense of urgency ("delivery failed," "avoid return to sender") and prompts the user to click on a link to update their address. This is a common tactic used by scammers to trick users into visiting malicious websites or providing personal information. The provided URL, "www.bestwebsite.com", is generic and doesn't resemble a legitimate shipping company's domain. The use of a generic domain and the urgent tone are significant red flags.
2. **Action**: Based on the urgency, the prompt to click a suspicious link, and the generic website address, this message is likely a scam.
3. **Final Answer**:
```json
{
  "label": "Scam",
  "reasoning": "The message employs urgency by stating a 'package delivery failed' and threatening 'return to sender'. It then directs the user to update their address via a generic-looking URL ('www.bestwebsite.com'), which is a common phishing technique. Legitimate delivery services usually provide specific tracking numbers and 

In [14]:
#6. Handle the output
def extract_json(text):
    """
    Finds the first '{' and the last '}' in a string 
    and parses it as JSON.
    """
    try:
        # Regex to find the JSON block inside the text
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
    except Exception as e:
        print(f"JSON Parsing Error: {e}")
    
    # Fallback if parsing fails
    return {
        "label": "Uncertain", 
        "reasoning": "Could not parse AI response",
        "intent": "Unknown",
        "risk_factors": ["Parsing Error"]
    }

paresed_output = extract_json(response.text)
print("*"*50)
print(paresed_output)
print("*"*50)
print(type(paresed_output))
print("*"*50)

**************************************************
{'label': 'Scam', 'reasoning': "The message employs urgency by stating a 'package delivery failed' and threatening 'return to sender'. It then directs the user to update their address via a generic-looking URL ('www.bestwebsite.com'), which is a common phishing technique. Legitimate delivery services usually provide specific tracking numbers and more credible website addresses.", 'intent': 'To phish for personal information (address, possibly payment details if further steps are required) or to redirect the user to a malicious website.', 'risk_factors': ['Urgency and threat of negative consequence (return to sender)', 'Generic and untrustworthy URL', 'Request to update personal information via a link', "Lack of specific details (e.g., tracking number, sender's name)"]}
**************************************************
<class 'dict'>
**************************************************
